In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import re

#Preprocesamiento
import spacy
from spacy.lang.es.stop_words import STOP_WORDS #importar set de stopwords
from nltk.stem import SnowballStemmer #importar stemmer
nlp = spacy.load('es_core_news_sm') #python -m spacy download es

#Bag-of-words
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import TfidfVectorizer

sns.set()

In [440]:
def tokenizer(doc, sep=None, stopwords_remove =False, lemmatization=False, stemming = False, vocabulary=None):
    '''
    Por defecto divide la sentencia por el carácter espacio.
    Ej: 'Data Mining is the best course'->['Data',  'Mining', 'is', 'the', 'best', 'course']
    
    Input: 
    1. doc: str, documento.
    2. sep: str, carácter para dividir el documento en tokens, por defecto es el espacio.
    3. stopwords_remove: bool, si es True remueve los stopwords del documento.
    4. lemmatization: bool, si es True lleva las palabras a su lema.
    5. stemming: bool, si es True lleva las palabas a su raíz.
    6. vocabuary: list, si un vocabulario es dado filtra las palabras que no estan presentes en el.
    
    Output: 
    list, lista de tokens.
    
    Nota: aplicar stemming y lemmatization al mismo tiempo no es correcto.
    '''
    doc = re.sub(r'\S+@\S+', '', doc) #elimina correos electrónicos  
    doc = re.sub(r'[^\w\s]','', doc) #elimina los símbolos de puntuación excepto underscore
    doc = re.sub(r'[_]', '', doc) #elimina underscore
    doc = re.sub(r'[a-zA-Z]+[0-9]+', '', doc) #elimina los tokens que contienen letras y números
    doc = re.sub(r'([ø ÿ þ])', ' ', doc) #reemplazas los símbolos contenidos por un espacio 
    doc = re.sub(r'[0-9]', '', doc) #elimina los tokens númericos
    doc = re.sub(r'(ñuñoa)', ' ñuñoa ', doc) #reemplaza ñuñoa por ñuñoa más espacios
    doc = re.sub(r'(daños)', ' daños ', doc) #reemplaza daños por daños más espacios
    tokens = doc.split(sep) #tokenización
    tokens = [word.lower() for word in tokens] #pasar todas las palabras a minúsculas
    
    
    
    if stopwords_remove ==True: #remover stopwords y palabras con menos de tres caracteres
        tokens = [word for word in tokens if word not in STOP_WORDS and len(word)>2]
    
    if lemmatization==True:
        tokens = [nlp(word)[0].lemma_ for word in tokens]
        
    if stemming == True:
        stemmer = SnowballStemmer('spanish')
        tokens = [stemmer.stem(word) for word in tokens]
    
    if vocabulary is not None:
        tokens = [word for word in tokens if word in vocabulary]
    
    return tokens


In [247]:
args = {"stopwords_remove": False, 'stemming': False}

In [441]:
#Creamos objetos para llevar el corpus a bag-of-words
tf_vectorizer = CountVectorizer(analyzer='word', tokenizer=lambda text: tokenizer(text, **args))
tfidf_vectorizer = TfidfVectorizer(analyzer='word', tokenizer=lambda text: tokenizer(text, **args)) 

In [420]:
#Importar la base de datos
df = pd.read_pickle('../data/robos_prose_v1.pkl')
df.head()

,sin_fecha_siniestro,sin_relato
id_prose,,
33474,2011-01-01,Por medio de la presente comunicamos a uds....
32730,2011-01-01,ESTABA ESTACIONADO EL LA CALLE ROTEMBURGO ENTR...
33469,2011-01-01,Estacionado al volver no encontre el vehiculo...
33466,2011-01-01,El abuelo de mi polola salio del domicilio y ...
32220,2011-01-01,ESTANDO ESTACIONADO VEHICULO EN AVDA PACIFICO ...


In [442]:
#extraer vocabulario
tf_vectorizer.fit(df['sin_relato'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function <lambda> at 0x0000026A02A6B6A8>,
                vocabulary=None)

In [443]:
#crear dataframe con las frecuencias de las palabras
freq_by_word = tf_vectorizer.vocabulary_
vocabulary = pd.Series(list(freq_by_word.keys()))
frequency = pd.Series(list(freq_by_word.values()))
df_freq_by_word = pd.DataFrame({'vocabulary':vocabulary, 'frequency':frequency})
df_freq_by_word.head()

,vocabulary,frequency
0,por,33913
1,medio,28291
2,de,12106
3,la,25266
4,presente,34442


In [450]:
df_freq_by_word.sort_values('frequency', ascending=False).head(30)

,vocabulary,frequency
44859,útiles,46310
11619,único,46309
5145,única,46308
26707,últimos,46307
6297,último,46306
16425,última,46305
34460,ú,46304
7984,ùltima,46303
26668,óvil,46302
4214,ósea,46301


In [451]:
df_freq_by_word.sort_values('frequency', ascending=False).tail(30)

,vocabulary,frequency
26448,abajn,29
35204,abajarme,28
31236,abajar,27
39017,abajao,26
27161,abaj,25
9105,abadonado,24
18750,abado,23
8963,abadia,22
32543,abad,21
8628,aba,20


Elementos extras a procesar:

1. Carácteres a extraer: [ø, ÿ, þ]
2. normalizar (stemming, lemmatization): [óptipcos, óptico]
3. Eliminar tíldes.
4. Split: [ñuñoa, daños]
5. vehiculo: [veh., vh, vehículo, vehiculo]

- Carácteres a homologar: [(ç, d)]
- stopwords
- corrección ortográfica usando embedding (útil para leves pifias, 60 ms).
- el otro corrector ortográfico tarda 230 ms.
- lemmatizador spacy -> 5 ms, lemmatizador CREA -> 137 ns, no funcinan con faltas ortográficas.
- ver que carácteres están cortando las palabras